In [1]:
import numpy as np
import pandas as pd
from astropy.table import Table, vstack
from matplotlib import pyplot as plt
from glob import glob
from astropy.stats import LombScargle
from scipy import stats
from scipy.optimize import curve_fit
import warnings
from TESStools import *
import celerite
from celerite import terms
from scipy.optimize import minimize
from tqdm.notebook import tqdm

In [2]:
fyps = pd.read_csv('fyps.csv')

In [8]:
header = ''
syms = ['^*','^\\dagger','^\\ddagger','^\\dagger^\\dagger','^\\ddagger^\\ddagger']
for i, star in tqdm(fyps.iterrows(),total=len(fyps)):
    
    fname = star['CommonName']
    print(fname)
    tic = star['TIC ID']
    
    lc, lc_smooth = lc_extract(get_lc_from_id(tic), smooth=128)
    time, flux, err = lc['Time'], lc['Flux'], lc['Err']
    
    good_fs, good_amps, good_phases, good_snrs, good_peaks = prewhiten_harmonic(time, flux, err)
 
    harmonics = harmonic_search(good_fs)
    
    bases = np.unique(harmonics[0])
    
    inds = [np.where(good_fs[:,0] == b)[0][0] for b in bases]
    
    bases = bases[np.argsort(inds)]
    
    sym_dict = {base:syms[i] for i,base in enumerate(bases)}
    f_dict = {syms[i]:base for i,base in enumerate(bases)}

    harmonics_flag = True
    
    combos = combo_search(good_fs)
    
    combo_footnote = ''
    for c in combos.T:
        i_0 = np.where(good_fs == c[0])[0][0]
        i_1 = np.where(good_fs == c[1])[0][0]
        i_2 = np.where(good_fs == c[2])[0][0]
        combo_footnote += f'$f_{i_2} = f_{i_0} + f_{i_1}$. '

    
    header += r"""\begin{{deluxetable*}}{{lcccccccc}}
\tabletypesize{{\footnotesize}}
\tablecaption{{Unique frequencies, amplitudes, phases, and formal errors for {0} found via prewhitening. For each frequency, we specify the SNR as defined in text, and the height of the associated peak in the RPS at that stage of prewhitening.\label{{tab:{0}_freqs}}}}
\tablehead{{\colhead{{Frequency}} & \colhead{{$f_j$}} & \colhead{{$\epsilon(f_j)$}} & \colhead{{$A_j$}} & \colhead{{$\epsilon(A_j)$}} & \colhead{{$\phi_j$}} &  \colhead{{$\epsilon(\phi_j)$}} & \colhead{{SNR}} & \colhead{{RPS Peak Height}}\\
\colhead{{}} & \colhead{{[day$^{{-1}}$]}} & \colhead{{[day$^{{-1}}$]}} & \colhead{{[ppt]}} & \colhead{{[ppt]}} & \colhead{{[radians]}} & \colhead{{[radians]}} & \colhead{{}} & \colhead{{}} }} 
\startdata
""".format(fname)
    for j,(f,a,p,s,pk) in enumerate(zip(good_fs,good_amps,good_phases,good_snrs,good_peaks)):
        row = f'$f_{{{int(j)}}}'
        if harmonics_flag:
            try:
                for b in harmonics[0][harmonics[1] == f[0]]:
                    row += sym_dict[b]
            except:
                harmonics_flag = False
        row += '$'
            
        header += row + f"& ${f[0]:.8f}$ & ${f[1]:.8f}$ & ${a[0]*1e3:.8f}$ & ${a[1]*1e3:.8f}$ & ${p[0]:.4f}$ & ${p[1]:.4f}$ & {s:.4f} & {pk:.4f} \\\\ \n".replace('e','\\times10^{')
    header.rstrip()
    header.rstrip('\\')
    header.rstrip()
    header += '\\enddata \n'
    if harmonics_flag:
        header += r'\tablecomments{'
        for k in f_dict.keys():
            f_ind = np.where(good_fs[:,0] == f_dict[k])[0][0]
            header += f'${k.replace("^","")}$: harmonics of $f_{{{f_ind}}}$. '
        header += combo_footnote
        header += '} \n'
    else:
        if len(combo_footnote) > 0:
            header += r'\tablecomments{'
            header += combo_footnote
            header += '} \n'

    header += '\\end{deluxetable*} \n\n'

    
    
with open("appendix_deluxetable.txt", "w") as text_file:
        text_file.write(header)

HD 269953
Found 4 frequencies
4 unique frequencies
HD 269110
Found 4 frequencies
4 unique frequencies
HD 268687
Found 6 frequencies
6 unique frequencies
HD 269840
Found 7 frequencies
7 unique frequencies
HD 269902
Found 3 frequencies
3 unique frequencies



In [9]:
len(combo_footnote)

0